In [1]:
from mylib.maze_utils3 import *
from NaiveBayesianDecoder import *
from StandardPCSimulation import *
from PolyFieldsPCSimulation import *
import time

import pandas as pd


def decoding_test(TestID, data_file, shuffle_name = ''):
    n = TestID
    totalpath = 'G:\YSY\Simulaiton_pc'
    mkdir(os.path.join(totalpath,'decoding_test'+shuffle_name))
    file = pd.read_excel(os.path.join(totalpath,'Decoding_Test_Records.xlsx'), sheet_name = 'Decoding_Test')

    file.loc[n-1,'Time begin'] = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(time.time()))
    if TestID > len(file):
        print("TestID wrong.")
        return

    print("  1. Initiate key parameters...")
    # initiate key parameters:      
    if np.isnan(file['nx'][n-1]):
        nx = 48   # default value = 48
    else:
        nx = int(file['nx'][n-1])
    
    if np.isnan(file['Testing Size'][n-1]):
        testing_size = 0.1
    else:
        testing_size = file['Testing Size'][n-1]
        
    training_size = 1-testing_size
        
    
    if file['Loss function type'][n-1] not in ['d','d2','0-1']:
        Loss = 'd2'
    else:
        Loss = str(file['Loss function type'][n-1])
    
    # Generate PCPopulations by simulating or read existed PCPopulation_test.pkl file
    print("  2. Generating  PCPopulation:")
    file.loc[n-1,'Data File'] = data_file
    if os.path.exists(file['Data File'][n-1]) == False:
        print("    ERROR! Do not file data file ",file['Data File'][n-1])
        return
    else:
        with open(data_file,'rb') as handle:
            trace = pickle.load(handle)
            
    T = trace['Spikes'].shape[1]
    n_neuorn = trace['n_neuron']
    maze_type = trace['maze_type']

    # Preprocessing the PCPopulation
    print("  3. Preprocessing the simulated place cells' data and split the complete data into training set and testing set.")
    Spikes = trace['Spikes']
    spike_nodes = trace['spike_nodes']
    
    training_size = int(T * training_size)
    rand_idx = np.array(range(0,training_size), dtype = np.int64)
    notrand_idx = np.array(range(training_size, Spikes.shape[1]), dtype = np.int64)
    Spikes_train = Spikes[:,rand_idx]
    MazeID_train = spike_nodes[rand_idx]
    Spikes_test = Spikes[:,notrand_idx]
    MazeID_test = spike_nodes[notrand_idx]
    
    print("  4. Fiting BayesModel...")
    BayesModel = NaiveBayesDecoder(res = nx, maze_type = maze_type, l = 0.01, Loss_function = Loss)
    BayesModel.fit(Spikes_train,MazeID_train)
    BayesModel.predict(Spikes_test,MazeID_test)
        
    # Assess the results.
    print("  5. Assessing the results...")
    AbD = BayesModel.metrics_MSE()
    print(AbD)
    HitRate = BayesModel.metrics_Accuracy()
    print(HitRate)
    
    with open(os.path.join(totalpath,'test_decoding'+shuffle_name,'PCPrediction_test'+str(TestID)+'.pkl'), 'wb') as f:
        pickle.dump(BayesModel,f)

    # Saving the test's information.
    print("  6. Saving test results...")
    file.loc[n-1,'Decoder Version'] = BayesModel.version
    file.loc[n-1,'Maze Type'] = maze_type
    file.loc[n-1,'MAE'] = BayesModel.MAE
    file.loc[n-1,'RMSE'] = BayesModel.RMSE
    file.loc[n-1,'absolute Accuracy'] = BayesModel.abHit
    file.loc[n-1,'Loss function type'] = Loss
    if nx != 12:
        file.loc[n-1, 'general Accuracy'] = BayesModel.geHit
    file.loc[n-1,'nx'] = nx
    file.loc[n-1,'Neuron Number'] = n_neuorn
    file.loc[n-1,'Total Frame Size'] = T
    file.loc[n-1,'Testing Size'] = testing_size
    file.loc[n-1,'Results File'] = os.path.join(totalpath,'test'+shuffle_name,'PCPrediction_test'+str(TestID)+'.pkl')
    file.loc[n-1,'Time end'] = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(time.time()))
    file.to_excel(os.path.join(totalpath,'Decoding_Test_Records_output.xlsx'), sheet_name = "Decoding_Test")
    print("Finish.")

totalpath = "G:\YSY\Cross_maze"
file_list = pd.read_excel(os.path.join(totalpath,"cross_maze_paradigm.xlsx"), sheet_name = "behavior") 
i = 70
loc = os.path.join(totalpath,str(int(file_list['MiceID'][i])), str(int(file_list['date'][i])),
                   'session '+str(int(file_list['session'][i])),'trace.pkl')
decoding_test(1, loc)

    G:\YSY\Simulaiton_pc\decoding_test is already existed!
  1. Initiate key parameters...
  2. Generating  PCPopulation:
  3. Preprocessing the simulated place cells' data and split the complete data into training set and testing set.
  4. Fiting BayesModel...
    Generate D matrix
    D matrix successfully generated!
    Generating tuning curve
  Tuning curve successfully generated!
    Generating P matirx...


100%|██████████████████████████████████████████████████████████████████████████████| 2571/2571 [01:16<00:00, 33.48it/s]


  5. Assessing the results...
MSE
29.888005656902312
(893.2928821470246, 1255.9518590038617, 29.888005656902312, 23.540143024452544)
Accuracy
(0.007390120575651498, 0.09101516919486581)
  6. Saving test results...


NameError: name 'data_type' is not defined